In [1]:
!pip install torch
!pip install LibRecommender

In [2]:
import numpy as np
import pandas as pd
from libreco.data import random_split, DatasetPure
from libreco.algorithms import NCF
from libreco.evaluation import evaluate


Instructions for updating:
non-resource variables are not supported in the long term


In [4]:
ratings = pd.read_csv(r"C:\Users\vish8\OneDrive\Desktop\Cursos\RecommendationSystem\RecommendationSystemsWithMLandDL\ml-latest-small\ml-latest-small\ratings.csv")
ratings.columns = ["user", "item", "label", "time"]

movies = pd.read_csv(r"C:\Users\vish8\OneDrive\Desktop\Cursos\RecommendationSystem\RecommendationSystemsWithMLandDL\ml-latest-small\ml-latest-small\movies.csv")
movies.columns = ["movieID", "Title", "Genres"]

In [5]:
training_set, evaluation_set, testing_set = random_split(ratings, multi_ratios=[0.8, 0.1, 0.1])

In [6]:
# Convert training, evaluation, and test data into format required by LibRecommender ("Pure" collobarative filtering data)
training_set, data_info = DatasetPure.build_trainset(training_set)
evaluation_set = DatasetPure.build_evalset(evaluation_set)
testing_set = DatasetPure.build_testset(testing_set)

In [7]:
ncf = NCF(
    task="rating",
    data_info=data_info
)

#Defaults used for hyperparams, see https://librecommender.readthedocs.io/en/latest/api/algorithms/ncf.html

In [8]:
# Train it

ncf.fit(
    training_set,
    neg_sampling=False, #False for explicit ratings, true for implicit, positive-only data
    verbose=2, # Print evaluation metrics
    eval_data=evaluation_set,
    metrics=["loss"],
)

Training start time: 2025-03-17 14:31:52


AttributeError: `batch_normalization` is not available with Keras 3.

In [8]:
# Test it

evaluate(
    model=ncf,
    data=testing_set,
    neg_sampling=False,
    metrics=["loss"], #for implicit, might use precision or recall
)

eval_pointwise: 100%|███████████████████████████| 2/2 [00:00<00:00, 250.07it/s]


{'loss': 1.032931}

In [9]:
# Print out every movie user 1 rated
filtered_rows = ratings[ratings.user == 1]

for index, row in filtered_rows.iterrows():
    rating = row['label']
    movie = movies[movies.movieID == row['item']]
    movieTitle = movie['Title']
    print(rating, movieTitle)

2.5 30    Dangerous Minds (1995)
Name: Title, dtype: object
3.0 833    Dumbo (1941)
Name: Title, dtype: object
3.0 859    Sleepers (1996)
Name: Title, dtype: object
2.0 906    Escape from New York (1981)
Name: Title, dtype: object
4.0 931    Cinema Paradiso (Nuovo cinema Paradiso) (1989)
Name: Title, dtype: object
2.0 1017    Deer Hunter, The (1978)
Name: Title, dtype: object
2.0 1041    Ben-Hur (1959)
Name: Title, dtype: object
2.0 1047    Gandhi (1982)
Name: Title, dtype: object
3.5 1083    Dracula (Bram Stoker's Dracula) (1992)
Name: Title, dtype: object
2.0 1087    Cape Fear (1991)
Name: Title, dtype: object
2.5 1111    Star Trek: The Motion Picture (1979)
Name: Title, dtype: object
1.0 1140    Beavis and Butt-Head Do America (1996)
Name: Title, dtype: object
4.0 1515    French Connection, The (1971)
Name: Title, dtype: object
4.0 1665    Tron (1982)
Name: Title, dtype: object
3.0 1708    Gods Must Be Crazy, The (1980)
Name: Title, dtype: object
2.0 1743    Willow (1988)
Name: Titl

In [10]:
# Predict rating of user 1 (who seems to like artsy dramas) of movie 1 (Toy Story)
ncf.predict(user=1, item=1)

array([2.7327979], dtype=float32)

In [11]:
# recommend 10 items for user 1
recs = ncf.recommend_user(user=1, n_rec=10)
print (recs)

{1: array([ 2023, 27773,  1856,   481, 54259,  7155,  1147,  3896,  3452,
       27611], dtype=int64)}


In [12]:
# Print what these are
myRecs = recs[1]
for rec in myRecs:
    print(movies[movies.movieID == rec].Title)

1585    Godfather: Part III, The (1990)
Name: Title, dtype: object
5941    Old Boy (2003)
Name: Title, dtype: object
1435    Kurt & Courtney (1998)
Name: Title, dtype: object
428    Kalifornia (1993)
Name: Title, dtype: object
6741    Stardust (2007)
Name: Title, dtype: object
5028    Calendar Girls (2003)
Name: Title, dtype: object
915    When We Were Kings (1996)
Name: Title, dtype: object
3112    Way of the Gun, The (2000)
Name: Title, dtype: object
2756    Romeo Must Die (2000)
Name: Title, dtype: object
5919    Battlestar Galactica (2003)
Name: Title, dtype: object
